In [ ]:
!pip3 install deeplake

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import deeplake

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from math import cos as cos
from math import sin as sin
from torch.utils.data import TensorDataset
import warnings
warnings.filterwarnings('ignore')

In [ ]:
load_train = deeplake.load('hub://activeloop/pascal-voc-2007-train-val')

In [ ]:
train_dataloader = load_train.pytorch(num_workers=0, batch_size=1, shuffle=False)

In [ ]:
def data_prep(train_data):

  train_imgs = []
  train_bb = []
  train_lab = []

  for i in range(len(train_data)):
    train_imgs.append(np.array(train_data[i]['images'][0]))
    train_bb.append(np.array(train_data[i]['boxes/box'][0]))
    train_lab.append(np.array(train_data[i]['boxes/label'][0] - 1))

  train_hsv = []
  for i in range(len(train_imgs)):
    train_imgs[i] = cv2.resize(train_imgs[i], (448, 448))
    train_hsv.append(cv2.cvtColor(train_imgs[i], cv2.COLOR_RGB2HSV))

  train_hsv = np.array(train_hsv)

  train_hsv = train_hsv.astype(np.float32)

  train_hsv = torch.from_numpy(train_hsv)

  train_ihsv = torch.clone(train_hsv).type(torch.complex64)

  train_hsv = train_hsv.permute(3, 0, 1, 2)
  train_ihsv = train_ihsv.permute(3, 0, 1, 2)

  train_ihsv[0] = torch.complex(train_hsv[2], train_hsv[1])
  train_ihsv[1] = torch.complex(train_hsv[1]*train_hsv[0], train_hsv[2])
  train_ihsv[2] = torch.complex(train_hsv[1]*torch.cos(train_hsv[0]), train_hsv[1]*torch.sin(train_hsv[0]))

  train_ihsv = train_ihsv.permute(1, 2, 3, 0)
  train_hsv = train_hsv.permute(1, 2, 3, 0)

  for bb in range(len(train_bb)):

    x_scale, y_scale = train_imgs[i].shape[1], train_imgs[i].shape[0]

    for val in range(len(train_bb[bb])):
      train_bb[bb][val][0] *= (448 / y_scale)
      train_bb[bb][val][1] *= (448 / x_scale)
      train_bb[bb][val][2] *= (448 / y_scale)
      train_bb[bb][val][3] *= (448 / x_scale)

  train_hsv = np.array(train_hsv)
  train_imgs = train_imgs.astype(np.float32)
  train_imgs = torch.from_numpy(train_imgs)

  return train_imgs, train_ihsv, train_bb, train_lab


In [ ]:
train_data = []
for id, X in enumerate(train_dataloader):
  if id >= 1001:
    train_data.append(X)
  if id >= 1500:
    break

In [ ]:
'''
0 -> 499
500 -> 999
1001 -> 1500
'''

In [ ]:
len(train_data)

In [ ]:
train_imgs, train_ihsv, train_bb, train_lab = data_prep(train_data)

In [ ]:
grid_size = 7
image_size = (64, 64)
num_BB = 2
num_classes = 20
depth = 5 * num_BB + num_classes
num_epoches = 2
Batch_size = 8

In [ ]:
def get_ground_truth(BB, Labels):

      grid_size_x = image_size[1] / grid_size
      grid_size_y = image_size[0] / grid_size

      boxes = {}
      class_names = {}
      ground_truth = torch.zeros((grid_size, grid_size, depth))

      for i in range(len(BB)):
          label = Labels[i]
          x_min, y_min, x_max, y_max = BB[i][0], BB[i][1], BB[i][2], BB[i][3]
          center = ((x_max + x_min) / 2, (y_max + y_min) / 2)
          col_index = int(center[0] // grid_size_x)
          row_index = int(center[1] // grid_size_y)
          if (row_index, col_index) not in boxes.keys():
            boxes[(row_index, col_index)] = 0
          if (0 <= col_index < grid_size and 0 <= row_index < grid_size and boxes[(row_index, col_index)]<num_BB):
                  ground_truth[row_index][col_index][2*5 + label] = 1.0
                  ground_truth[row_index][col_index][boxes[(row_index, col_index)]*5] = 1.0
                  ground_truth[row_index][col_index][boxes[(row_index, col_index)]*5 + 1] = (center[0] - col_index * grid_size_x) / image_size[0]
                  ground_truth[row_index][col_index][boxes[(row_index, col_index)]*5 + 2] = (center[1] - row_index * grid_size_y) / image_size[1]
                  ground_truth[row_index][col_index][boxes[(row_index, col_index)]*5 + 3] = (x_max - x_min) / image_size[0]
                  ground_truth[row_index][col_index][boxes[(row_index, col_index)]*5 + 4] = (y_max - y_min) / image_size[1]

                  boxes[(row_index, col_index)] += 1
      return ground_truth.tolist()

In [ ]:
ground_truth = []
for i in range(len(train_bb)):
  ground_truth.append(get_ground_truth(train_bb[i], train_lab[i]))
ground_truth = torch.Tensor(ground_truth)

In [ ]:
import os

directory_path = '/content/drive/MyDrive/VOC2007/Train/Batch3'

if not os.path.exists(directory_path):
    os.makedirs(directory_path)
    print(f"Directory '{directory_path}' created.")
else:
    print(f"Directory '{directory_path}' already exists.")

imgs_path = os.path.join(directory_path, 'train_imgs.pt')
if not os.path.exists(imgs_path):
    torch.save(train_imgs, imgs_path)
    print(f"Tensor saved to '{imgs_path}'.")
else:
    print(f"File '{imgs_path}' already exists. Not saving the tensor.")

ihsv_path = os.path.join(directory_path, 'train_ihsv.pt')
if not os.path.exists(ihsv_path):
    torch.save(train_ihsv, ihsv_path)
    print(f"Tensor saved to '{ihsv_path}'.")
else:
    print(f"File '{ihsv_path}' already exists. Not saving the tensor.")

# bb_path = os.path.join(directory_path, 'train_bb.pt')
# if not os.path.exists(bb_path):
#     torch.save(train_bb, bb_path)
#     print(f"Tensor saved to '{bb_path}'.")
# else:
#     print(f"File '{bb_path}' already exists. Not saving the tensor.")


# lab_path = os.path.join(directory_path, 'train_lab.pt')
# if not os.path.exists(lab_path):
#     torch.save(train_lab, lab_path)
#     print(f"Tensor saved to '{lab_path}'.")
# else:
#     print(f"File '{lab_path}' already exists. Not saving the tensor.")

gt_path = os.path.join(directory_path, 'train_gt.pt')
if not os.path.exists(gt_path):
    torch.save(ground_truth, gt_path)
    print(f"Tensor saved to '{gt_path}'.")
else:
    print(f"File '{gt_path}' already exists. Not saving the tensor.")

In [ ]:
val_data = []
for id, X in enumerate(train_dataloader):
  if id >= 1501:
    val_data.append(X)
  if id >= 1600:
    break

In [ ]:
len(val_data)

In [ ]:
val_imgs, val_ihsv, val_bb, val_lab = data_prep(val_data)

In [ ]:
ground_truth = []
for i in range(len(val_bb)):
  ground_truth.append(get_ground_truth(val_bb[i], val_lab[i]))
ground_truth = torch.Tensor(ground_truth)

In [ ]:
import os

directory_path = '/content/drive/MyDrive/VOC2007/Validation/Batch1'

if not os.path.exists(directory_path):
    os.makedirs(directory_path)
    print(f"Directory '{directory_path}' created.")
else:
    print(f"Directory '{directory_path}' already exists.")

imgs_path = os.path.join(directory_path, 'val_imgs.pt')
if not os.path.exists(imgs_path):
    torch.save(val_imgs, imgs_path)
    print(f"Tensor saved to '{imgs_path}'.")
else:
    print(f"File '{imgs_path}' already exists. Not saving the tensor.")

ihsv_path = os.path.join(directory_path, 'val_ihsv.pt')
if not os.path.exists(ihsv_path):
    torch.save(val_ihsv, ihsv_path)
    print(f"Tensor saved to '{ihsv_path}'.")
else:
    print(f"File '{ihsv_path}' already exists. Not saving the tensor.")

# bb_path = os.path.join(directory_path, 'val_bb.pt')
# if not os.path.exists(bb_path):
#     torch.save(val_bb, bb_path)
#     print(f"Tensor saved to '{bb_path}'.")
# else:
#     print(f"File '{bb_path}' already exists. Not saving the tensor.")


# lab_path = os.path.join(directory_path, 'val_lab.pt')
# if not os.path.exists(lab_path):
#     torch.save(val_lab, lab_path)
#     print(f"Tensor saved to '{lab_path}'.")
# else:
#     print(f"File '{lab_path}' already exists. Not saving the tensor.")

gt_path = os.path.join(directory_path, 'val_gt.pt')
if not os.path.exists(gt_path):
    torch.save(ground_truth, gt_path)
    print(f"Tensor saved to '{gt_path}'.")
else:
    print(f"File '{gt_path}' already exists. Not saving the tensor.")